在Python中执行整数和浮点数的数学运算时很简单的。  尽管如此，如果你需要
# 执行分数、数组或者是日期和时间的运算的话，就得做更多的工作了。本章集中讨论的就是这些主题。

## 3.1 数字的四舍五入

想对浮点数执行精度的舍入运算,对于简单的舍入运算使用内置的`round`函数即可

In [2]:
print(round(1.23, 1)) #保留1位
round(1.23561, 3)

1.2


1.236

当一个值刚好在两个边界的中间的时候， round 函数返回离它最近的偶数。也就 是说，对 1.5 或者 2.5 的舍入运算都会得到 2。
传给 round() 函数的 ndigits 参数可以是负数，这种情况下，舍入运算会作用在 十位、百位、千位等上面。比如
```python
a = 1627730
round(a, -2) #1627770 ， 作用在百位上
```

**不要将舍入和格式化输出搞混淆**了。如果你的目的只是简单的输出一定宽度的数， 你不需要使用 round() 函数。  
而仅仅只需要在格式化的时候指定精度即可。比如

In [6]:
x = 1.23456
print(format(x, '0.2f')) #保留2位
'value is {:0.3f}'.format(x)  #冒号后面的.f表示将该参数格式化为浮点数，并保留f位小数

1.23


'value is 1.235'

同样，不要试着去舍入浮点值来” 修正” 表面上看起来正确的问题。比如，你可能 倾向于这样做：

In [12]:
a = 2.1 
b = 4.2
c = a+b
print(c)
c = round(c, 2) #企图修复
c

6.300000000000001


6.3

对于大多数使用到浮点的程序，没有必要也不推荐这样做。  
尽管在计算的时候会有 一点点小的误差，但是这些小的误差是能被理解与容忍的。如果不能允许这样的小误 差 (比如涉及到金融领域)，  
那么就得考虑使用 decimal 模块了，下一节我们会详细讨 论。

## 3.2执行精确的浮点数运算

需要对浮点数执行精确的计算操作，并且不希望有任何小误差的出现  
浮点数的一个普遍问题是它们并不能精确的表示十进制数。并且，即使是最简单的 数学运算也会产生小的误差，比如上述`a+b == 6.3 会返回False`

这些错误是由底层 CPU 和 IEEE 754 标准通过自己的浮点单位去执行算术时的特征。  
由于 Python 的浮点数据类型使用底层表示存储数据，因此你没办法去避免这样的误差。  
如果你想更加精确 (并能容忍一定的性能损耗)，你可以使用 decimal 模块：

In [12]:
from decimal import Decimal
a = Decimal('2.1')
b = Decimal('4.2')
a+b

Decimal('6.3')

初看起来， 上面的代码好像有点奇怪，比如我们用字符串来表示数字。然而 Decimal 对象会像普通浮点数一样的工作 (支持所有的常用数学运算)。  
如果你打印它们或者在字符串格式化函数中使用它们，看起来跟普通数字没什么两样。  
decimal模块的一个主要特征是允许你控制计算的每一方面，包括数字位数和四舍五入运算。为了这样做，你先得创建一个本地上下文并更改它的设置，比如：

In [15]:
a = Decimal('1.3')
b = Decimal('1.7')
print( a / b)

0.7647058823529411764705882353


In [17]:
from decimal import localcontext
with localcontext() as ctx: 
    ctx.prec = 3 # 四舍五入 保留3位
    print(a / b)

0.765


decimal 模块实现了 IBM 的” 通用小数运算规范”。不用说，有很多的配置选项这 本书没有提到。

Python 新手会倾向于使用 decimal 模块来处理浮点数的精确运算。然而，先理解 你的应用程序目的是非常重要的。如果你是在做科学计算或工程领域的计算、电脑绘 图，或者是科学领域的大多数运算，那么使用普通的浮点类型是比较普遍的做法。其 中一个原因是， **在真实世界中很少会要求精确到普通浮点数能提供的 17 位精度**。 因 此，计算过程中的那么一点点的误差是被允许的。第二点就是，原生的浮点数计算要 快的多 有时候你在执行大量运算的时候速度也是非常重要的。

即便如此，你却不能完全忽略误差。数学家花了大量时间去研究各类算法，有些处 理误差会比其他方法更好。你也得注意下减法删除 已经大数和小数的加分运算所带来 的影响。比如：

In [18]:
nums = [1.23e+18, 1, -1.23e+18]
sum(nums) # Notice how 1 disappears  ？？

0.0

上面的错误可以利用 math.fsum() 所提供的更精确计算能力来解决：

In [19]:
import math
math.fsum(nums)

1.0

然而，对于其他的算法，你应该仔细研究它并理解它的误差产生来源。  
总的来说， decimal 模块主要用在涉及到金融的领域。在这类程序中，哪怕是一点小小的误差在计算过程中蔓延都是不允许的。  
因此， decimal 模块为解决这类问题提供了方法。当 Python 和数据库打交道的时候也通常会遇到 Decimal 对象，并且，通常也是在处理金融数据的时候

## 3.3数字的格式化输出
问题：你需要将数字格式化后输出， 并控制数字的位数、对齐、千位分隔符和其他的细节。
格式化输出单个数字的时候，可以使用内置的`format`

x = 1234.56789
```python
# Two decimal places of accuracy
format(x, '0.2f')  #'1234.57'

# Right justified in 10 chars, one-digit accuracy

format(x, '>10.1f') #' 1234.6'

# Left justified

format(x, '<10.1f') #'1234.6 '

 # Centered

format(x, '^10.1f') #' 1234.6 '

 # Inclusion of thousands separator 包含千位分隔符

format(x, ',') #'1,234.56789'

format(x, '0,.1f') #'1,234.6'
```

如果你想使用指数记法，将 f 改成 e 或者 E(取决于指数输出的大小写形式)。比如：

format(x, 'e')    #'1.234568e+03'  
format(x, '0.2E')  #'1.23E+03'  
同时指定宽度和精度的一般形式是 '[<>ˆ]?width[,]?(.digits)?' ，其中 width 和 digits 为整数，  
？代表可选部分。同样的格式也被用在字符串的 format() 方法中。 比如：  
`The value is {:0,.2f}'.format(x)`   # 'The value is 1,234.57'

## 3.4 二 八 十六进制整数
问题：你需要转换或者输出使用二进制，八进制或十六进制表示的整数。  
解决方案：为了将整数转换为二进制、八进制或十六进制的文本串，可以分别使用 `bin()` , `oct()` 或 `hex()` 函数：

In [20]:
x = 17
print(bin(x))  #不想要0b之类的前缀 可以使用format函数，如 format(x, 'b')
print(oct(x))
print(hex(x))

0b10001
0o21
0x11


整数是有符号的，所以如果你在处理负数的话，输出结果会包含一个负号。  
如果你想产生一个无符号值，你需要增加一个指示最大位长度的值。比如为了显示 32 位的值，可以像下面这样写：

In [21]:
x = -1234
format(2**32 + x, 'b')

'11111111111111111111101100101110'

为了以不同的**进制 转换 整数字符串**，简单的使用带有进制的 int() 函数即可：

In [24]:
int('10011010010', 2)

1234

大多数情况下处理二进制、八进制和十六进制整数是很简单的。只要记住这些转换 属于整数和其对应的文本表示之间的转换即可。永远只有一种整数类型  
最后，使用八进制的程序员有一点需要注意下。Python 指定八进制数的语法跟其 他语言稍有不同, `os.chmod('xxx.sh', 0o755)`  八进制的前缀是0o

## 3.5字节到大整数的打包与解包  
问题:你有一个字节字符串并想将它解压成一个整数。或者，你需要将一个大整数转换为 一个字节字符串。  
假设你的程序需要处理一个拥有 128 位长的 16 个元素的字节字符串。比如：

In [16]:
xdata = b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004' # \x 值为16位16进制的字符
print(len(xdata))
print(xdata)

16
b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'


为了将 bytes 解析为整数，使用 int.from bytes() 方法，并像下面这样指定字节 顺序

In [30]:
print(int.from_bytes(xdata, 'little'))
print(int.from_bytes(xdata, 'big'))

69120565665751139577663547927094891008
94522842520747284487117727783387188


为了将一个大整数转换为一个字节字符串，使用 int.to bytes() 方法，并像下面 这样指定字节数和字节顺序：


In [17]:
x = 94522842520747284487117727783387188
print(x.to_bytes(16, 'big'))
x.to_bytes(16, 'little')

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'


b'4\x00#\x00\x01\xef\xcd\x00\xab\x90x\x00V4\x12\x00'

大整数和字节字符串之间的转换操作并不常见。然而，在一些应用领域有时候也会 出现，比如密码学或者`网络`。例如，`IPv6 网络地址使用一个 128 位的整数表示。如果 你要从一个数据记录中提取这样的值的时候`，你就会面对这样的问题。  
作为一种替代方案，你可能想使用 6.11 小节中所介绍的 struct 模块来解压字节。 这样也行得通，不过利用 struct 模块来解压对于整数的大小是有限制的。因此，你可 能想解压多个字节串并将结果合并为最终的结果，就像下面这样：

In [32]:
import struct
hi, lo = struct.unpack('>QQ', xdata)
(hi << 64) + lo

94522842520747284487117727783387188

字节顺序规则 (little 或 big) 仅仅指定了构建整数时的字节的低位高位排列方式。  
 我们从下面精心构造的 16 进制数的表示中可以很容易的看出来：

In [34]:
x = 0x01020304
print(x.to_bytes(4, 'big'))
print(x.to_bytes(4, 'little'))

b'\x01\x02\x03\x04'
b'\x04\x03\x02\x01'


如果你试着将一个整数打包为字节字符串， 那么它就不合适了， 你会得到一个错 误。  
如果需要的话，你可以使用 int.bit length() 方法来决定需要多少字节位来存储 这个值。

## 3.6 复数的数学运算
问题：你写的最新的网络认证方案代码遇到了一个难题，并且你唯一的解决办法就是使用复数空间。  
再或者是你仅仅需要使用复数来执行一些计算操作。  
复数可以用使用函数 complex(real, imag)或者是带有后缀j的浮点数来指定。 

In [36]:
a = complex(2, 4)
b = 3 - 5j
print(a, b)

(2+4j) (3-5j)


对应的实部、虚部和共轭复数可以很容易的获取。就像下面这样：

In [38]:
print(a.real, a.imag)
a.conjugate()

2.0 4.0


(2-4j)

Python 中大部分与数学相关的模块都能处理复数。比如如果你使用 numpy，可以 很容易的构造一个复数数组并在这个数组上执行各种操作：

In [20]:
import numpy as np
a = np.array([2+3j, 4+5j, 6-7j, 8+9j])
a

array([2.+3.j, 4.+5.j, 6.-7.j, 8.+9.j])

In [22]:
print(a+2)
np.sin(a)

[ 4.+3.j  6.+5.j  8.-7.j 10.+9.j]


array([   9.15449915  -4.16890696j,  -56.16227422 -48.50245524j,
       -153.20827755-526.47684926j, 4008.42651446-589.49948373j])

## 3.7 无穷大与 NaN
问题：你想创建或测试正无穷、负无穷或 NaN(非数字) 的浮点数。  
Python 并没有特殊的语法来表示这些特殊的浮点值，但是可以使用 float() 来创 建它们。

In [27]:
a = float('inf')
b = float('-inf')
c = float('nan')
type(a)

float

In [37]:
'{0}, {1}'.format(1.1 > a, 1 > b)
##  测试这些值的存在，使用 math.isinf() 和 math.isnan() 函数
# math.isinf(a)
# math.isnan(c)

'False, True'

有一些地方需要你特别注意，特别是跟**比较和操作符**相关的时候  
- 无穷大数在执行数学计算的时候会传播
- NaN 值会在所有操作中传播，而不会产生异常

In [39]:
# 无穷大数在执行数学计算的时候会传播
print(a + 45)
# NaN 值会在所有操作中传播，而不会产生异常
print(c + 45)

inf
nan


NaN值的一个特别的地方时它们之间的比较操作总是返回 False
由于这个原因，测试一个 NaN 值得唯一安全的方法就是使用 math.isnan(),也就是上面演示的那样。  
有时候程序员**想改变 Python 默认行为，在返回无穷大或 NaN 结果的操作中抛出异常**。   
fpectl 模块可以用来改变这种行为，但是它在标准的 Python 构建中并没有被 启用，它是平台相关的，并且针对的是专家级程序员。可以参考在线的 Python 文档获 取更多的细节。

In [40]:
# NaN值的一个特别的地方时它们之间的比较操作总是返回 False
d = float('nan')
c ==d 

False

## 3.8 分数运算
问题: 你进入时间机器，突然发现你正在做小学家庭作业，并涉及到分数计算问题。或者你可能需要写代码去计算在你的木工工厂中的测量值。  
`fractions`模块可以被用来执行包含分数的数学运算。

In [42]:
from fractions import Fraction
a = Fraction(5, 4)
b = Fraction(7, 16)
print(a + b)

print(a.numerator) #分子
print(a.denominator) #分母
float(a) 

27/16
5
4


1.25

In [44]:

# 浮点数转换为分数
x = 3.75
y = Fraction(*x.as_integer_ratio())
y

Fraction(15, 4)

在大多数程序中一般不会出现分数的计算问题， 但是有时候还是需要用到的。 比如，在一个允许接受分数形式的测试单位并以分数形式执行运算的程序中，直接使用 分数可以减少手动转换为小数或浮点数的工作。

## 3.9大型数组运算
问题：你需要在大数据集 (比如数组或网格) 上面执行计算  
涉及到数组的重量级运算操作，可以使用 NumPy 库。   
NumPy 的一个主要特征是它会给 Python 提供一个数组对象， 相比标准的 Python 列表而已更适合用来做数学运算。下面是一个简单的小例子，向你展示标准列表对象和 NumPy 数组对象之间的差别：

In [50]:
x = [1, 2, 3, 4]
y = [5, 6, 7, 8]
print(x * 2) #并不是组内的数运算
print(x + y) #数组的拼接
x + 10 #错误

[1, 2, 3, 4, 1, 2, 3, 4]
[1, 2, 3, 4, 5, 6, 7, 8]


TypeError: can only concatenate list (not "int") to list

In [49]:
ax = np.array(x)
ay = np.array(y)
print(ax * 2, ax +10)

[2 4 6 8] [11 12 13 14]


正如所见，两种方案中数组的基本数学运算结果并不相同。  
特别的， NumPy 中的标 量运算 (比如 ax * 2 或 ax + 10 ) **会作用在每一个元素上**。  
另外，当两个操作数都是数组的时候执行元素对等位置计算，并最终生成一个新的数组。

对整个数组中**所有元素同时执行数学**运算可以使得作用在整个数组上的函数运算简单而又快速。比如，如果你想计算多项式的值，可以这样做

In [58]:
def f(x):   ## 传入np.array
    return 3*x**2 - 2*x + 1
f(ax)

array([ 2,  9, 22, 41])

numPy 还为数组操作提供了大量的通用函数，这些函数可以作为 math 模块中类似 函数的替代  
```np.sqrt(ax)```  
`使用这些通用函数要比循环数组并使用 math 模块中的函数执行计算要快的多`。因此，只要有可能的话尽量选择 NumPy 的数组方案。

底层实现中， NumPy 数组使用了 C 或者 Fortran 语言的机制分配内存。也就是说， 它们是一个非常大的连续的并由同类型数据组成的内存区域。所以，你可以构造一个比普通 Python 列表大的多的数组。比如，如果你想构造一个 10,000*10,000 的浮点数二维网格，很轻松

In [60]:
grid = np.zeros(shape=(10000,10000), dtype=float)
grid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
# 所有的普通操作还是会同时作用在所有元素上
grid + 10

array([[10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       ...,
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.]])

关于 NumPy 有一点需要特别的主意，那就是它扩展 Python 列表的索引功能 - 特别 是对于多维数组。为了说明清楚，先构造一个简单的二维数组并试着做些试验

In [73]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
print(a)

print(a[0]) # 选择第1行
print(a[:, 1]) #选择第2列

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
[1 2 3 4]
[ 2  6 10]


In [74]:
print(a + [10, 100, 1000, 10000]) #在所有行的操作中 广播行向量

[[   11   102  1003 10004]
 [   15   106  1007 10008]
 [   19   110  1011 10012]]


In [ ]:
np.where(a < 10, a, 10) #在数组上的条件赋值

## 3.10 矩阵与线性代数运算
问题：你需要执行矩阵和线性代数运算，比如矩阵乘法、寻找行列式、求解线性方程组等。  
NumPy 库有一个矩阵对象可以用来解决这个问题。
矩阵类似于 3.9 小节中数组对象，但是遵循线性代数的计算规则。下面的一个例子 展示了矩阵的一些基本特性

In [77]:
m = np.matrix([[1,-2,3],[0,4,5],[7,8,-9]])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
m

matrix([[ 1, -2,  3],
        [ 0,  4,  5],
        [ 7,  8, -9]])

In [78]:
m.T #转置

matrix([[ 1,  0,  7],
        [-2,  4,  8],
        [ 3,  5, -9]])

## 3.11 随机选择
问题：你想从一个序列中随机抽取若干元素，或者想生成几个随机数。  
`random`模块有大量的函数用来产生随机数和随机选择元素。比如，要想从一个序列中随机的抽取一个元素，可以使用 `random.choice()`

In [79]:
import random
values = [1, 2, 3, 4, 5, 6]
random.choice(values)

1

In [81]:
random.choice(values)

4

为了提取出 N 个不同元素的样本用来做进一步的操作，可以使用 `random.sample()`

In [82]:
random.sample(values, 2)

[3, 5]

打乱序列中元素的顺序，可以使用`random.shuffle()`  
生成随机整数，使用`random.randint()`

In [87]:
random.randint(0,10)
#生成 0 到 1 范围内均匀分布的浮点数，使用 random.random()

8

random 模块使用 Mersenne Twister 算法来计算生成随机数。  
这是一个**确定性算法**， 但是你可以通过`random.seed()`函数修改初始化种子。比如
```python
random.seed() # Seed based on system time or os.urandom() 
random.seed(12345) # Seed based on integer given
```
除了上述介绍的功能，random 模块还包含基于均匀分布、高斯分布和其他分布的 随机数生成函数。  
比如， `random.uniform() `计算均匀分布随机数， `random.gauss()` 计算正态分布随机数。对于其他的分布情况请参考在线文档。
在 random 模块中的函数不应该用在和密码学相关的程序中。如果你确实需要类似 的功能，可以使用 ssl 模块中相应的函数。比如， ssl.RAND bytes() 可以用来生成一 个安全的随机字节序列。

## 3.12 基本的日期与时间转换
问题:你需要执行简单的时间转换，比如天到秒，小时到分钟等的转换。  
为了执行不同时间单位的转换和计算，请使用 datetime 模块。  
如果你想表示指定的日期和时间，先创建一个 datetime 实例然后使用标准的数学 运算来操作它们。比如：

In [114]:
import time
from datetime import timedelta,date
from datetime import datetime
#a = timedelta(days=2, hours=6)

In [101]:
a = datetime(2012, 9, 23)
print(a + timedelta(days=10))

2012-10-03 00:00:00


In [102]:
b = datetime(2012, 12, 21)
d = b - a
d.days      #算间隔天数，且datetime 会自动处理闰年

89

In [107]:
now = datetime.today()          #注：  time.time()返回的是unix时间
print(now)
print(now + timedelta(minutes=30))      

2023-11-03 11:11:04.272090
2023-11-03 11:41:04.272090


对大多数基本的日期和时间处理问题， datetime 模块以及足够了。如果你需要执 行更加复杂的日期操作，比如处理时区，模糊时间范围，节假日计算等等，可以考虑 使用`dateutil`模块
许多类似的时间计算可以使用 dateutil.relativedelta() 函数代替。但是，有一 点需要注意的就是，它会在处理月份 (还有它们的天数差距) 的时候填充间隙。见PDF

## 3.13 计算最后一个周五的日期
问题：你需要查找星期中某一天最后出现的日期，比如星期五  
算法原理是这样的：先将开始日期和目标日期映射到星期数组的位置上 (星 期一索引为 0)，  
然后通过模运算计算出目标日期要经过多少天才能到达开始日期。  
然后用开始日期减去那个时间差即得到结果日期。

In [112]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def get_previous_byday(dayname, start_date=None):
    if start_date is None:
        start_date = datetime.today() 

    day_num = start_date.weekday() #该日期是星期几
    day_num_target = weekdays.index(dayname) #找到匹配value的第一个所以

    days_ago = (7 + day_num - day_num_target) % 7 
    if days_ago == 0:
        days_ago = 7 
    target_date = start_date - timedelta(days=days_ago) 
    return target_date

In [113]:
get_previous_byday('Monday')

datetime.datetime(2023, 10, 30, 14, 22, 28, 40860)

如果你要像这样执行大量的日期计算的话，你最好安装第三方包 python-dateutil 来代替。比如，使用 `dateutil`模块中的 relativedelta() 函数执行同样的计算

## 3.14 计算当前月份的日期范围
问题：你的代码需要在当前月份中循环每一天， 想找到一个计算这个日期范围的高效方法。

在这样的日期上循环并需要事先构造一个包含所有日期的列表。你可以先计算出开始日期和结束日期，   
然后在你步进的时候使用 datetime.timedelta 对象递增这个日期变量即可。  
下面是一个接受任意 datetime 对象并**返回一个由当前月份开始日和下个月开始日 组成的元组对象**。

In [115]:
import calendar
def get_month_range(start_date=None):
    if start_date is None:
        start_date = date.today().replace(day=1)

    _, days_in_month = calendar.monthrange(start_date.year, start_date.month) 
    end_date = start_date + timedelta(days=days_in_month)
    return (start_date, end_date)

In [122]:
day = timedelta(days=1)
first_day, last_day = get_month_range()
while first_day < last_day: 
    print(first_day)
    first_day += day

上面的代码先计算出一个对应月份第一天的日期。一个快速的方法就是使用 date 或 `datetime 对象的 replace()` 方法简单的将 days 属性设置成 1 即可。  
replace() 方法一个好处就是它会创建和你开始传入对象类型相同的对象。所以，如果输入参数 是一个 date 实例，那么结果也是一个 date 实例。同样的，如果输入是一个 datetime 实例，那么你得到的就是一个 datetime 实例。  
然后，使用`calendar.monthrange()` 函数来找出该月的总天数。任何时候只要你 **想获得日历信息，那么 calendar 模块**就非常有用了。 monthrange() 函数会返回包含星期和该月天数的元组。

为了在日期范围上循环， 要使用到标准的数学和比较操作。 比如， 可以利用 timedelta 实例来递增日期，小于号< 用来检查一个日期是否在结束日期之前。
理想情况下，如果能为日期迭代创建一个同内置的 range() 函数一样的函数就好 了。幸运的是，可以使用一个生成器来很容易的实现这个目标：

In [ ]:
def date_range(start, stop, step):
    while start < stop:
        yield start 
        start += step 

In [ ]:
for d in date_range(datetime(2012, 9, 1), datetime(2012,10,1), timedelta(hours=6)):
    print(d)

## 3.15 字符串转换为日期
问题 : 你的应用程序接受字符串格式的输入，但是你想将它们转换为 datetime 对象以便 在上面执行非字符串操作。  
使用 Python 的标准模块 datetime 可以很容易的解决这个问题

In [128]:
text = '2023-10-20'
y = datetime.strptime(text, '%Y-%m-%d')
x = datetime.now()
(x - y).days

14

datetime.strptime() 方法支持很多的格式化代码，比如 %Y 代表 4 位数年份， %m 代表两位数月份。  
还有一点值得注意的是这些格式化占位符也可以反过来使用，将日期输出为指定的格式字符串形式。  
比如，假设你的代码中生成了一个 datetime 对象，你想将它格式化为漂亮易读形 式后放在自动生成的信件或者报告的顶部,`strftime`

还有一点需要注意的是， strptime() 的性能要**比你想象中的差很多**，因为它是使 用纯 Python 实现，并且必须处理所有的系统本地设置。  
如果你要在代码中需要解析大量的日期并且**已经知道了日期字符串的确切格式，可以自己实现一套解析方案来获取 更好的性能**。  
比如，如果你已经知道所以日期格式是 YYYY-MM-DD ，你可以像下面这样 实现一个解析函数

In [129]:
def parse_ymd(s):
    year_s, mon_s, day_s = s.split('-') 
    return datetime(int(year_s), int(mon_s), int(day_s))

In [130]:
parse_ymd('2019-05-14')

datetime.datetime(2019, 5, 14, 0, 0)

## 3.16 结合时区的日期操作
问题:你有一个安排在 2012 年 12 月 21 日早上 9:30 的电话会议，地点在芝加哥。  
而你的 朋友在印度的班加罗尔，那么他应该在当地时间几点参加这个会议呢？

对几乎所有涉及到时区的问题，你都应该使用 `pytz `模块。这个包提供了 Olson 时 区数据库，它是时区信息的事实上的标准，在很多语言和操作系统里面都可以找到  
pytz 模块一个主要用途是将 datetime 库创建的简单日期对象本地化。比如，下面 如何表示一个芝加哥时间的示例

In [133]:
from pytz import timezone
d = datetime(2023, 11, 1, 9, 30, 0)
print(d)

2023-11-01 09:30:00


In [136]:
central = timezone('US/Central')   #Localize the date for Chicago
loc_d = central.localize(d)
print(loc_d)

2023-11-01 09:30:00-05:00


一旦日期被本地化了，它就可以转换为其他时区的时间了。为了得到班加罗尔对应的时间，你可以这样做

In [137]:
bang_d = loc_d.astimezone(timezone('Asia/Kolkata'))
print(bang_d)

2023-11-01 20:00:00+05:30


为了不让你被这些东东弄的晕头转向，处理本地化日期的通常的策略先将所有日期 转换为 UTC 时间，并用它来执行所有的中间存储和操作。比如

In [138]:
import pytz
utc_d = loc_d.astimezone(pytz.utc)
print(utc_d)

2023-11-01 14:30:00+00:00


注：当你阅读到这里的时候，有可能 pytz 模块以及不再建议使用了，因为 PEP431 提出了更先进的时区支持。 但是这里谈到的很多问题还是有参考价值的 (比如使用 UTC 日期的建议等)